# Importamos liberías

In [1]:
import datetime as dt
import tm_func as tm
import pyodbc
import warnings
import pandas as pd
import numpy as np
import os
from win32com.client import Dispatch

# Obtención de información general de gammas (queries del server)

In [2]:
server = 'nahualac.database.windows.net,1433' 
database = 'nahualac' 
username = 'Telematica' 
password = 'Telematic@2021' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

query = '''SELECT
        d.identificador dispositivo, CONCAT(prod.nombre, ' - ', mod.nombre) modelo, cat.nombre estatus, div.nombre division, z.nombre zona, c.clave proyecto
FROM
        tbl_dispositivo d
INNER JOIN
            tbl_gamma gg
                ON d.id = gg.dispositivoId
INNER JOIN
            tbl_estatusFisicos eF
                ON d.id = eF.dispositivoId
INNER JOIN
            cat_estatusFisicos cat
                ON eF.estatusFisicoId = cat.id
INNER JOIN
            tbl_modelo mod
                ON d.modeloId = mod.id
INNER JOIN
            cat_producto prod
                ON mod.productoId = prod.id
INNER JOIN
            tbl_grupo g
                ON d.grupoId = g.id
INNER JOIN
            tbl_zona z
                ON g.zonaId = z.id
INNER JOIN
            tbl_contrato c
                ON z.contratoId = c.id
INNER JOIN
            tbl_division div on c.divisionId = div.id;'''

## Modelo, division, zona y proyecto para cada Gamma

In [3]:
with warnings.catch_warnings(): 
    warnings.simplefilter('ignore', UserWarning) 
    data = pd.read_sql(query, cnxn)

data = data[["dispositivo","estatus","modelo","division","zona","proyecto"]]
data

,dispositivo,estatus,modelo,division,zona,proyecto
0,444464,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3
1,4442C3,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3
2,44431E,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3
3,444384,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3
4,44BB80,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3
...,...,...,...,...,...,...
11047,ICE006,INSTALADO,GAMMA - SIM,DEMO,DEMO,GAMMA DEMO
11048,ICE007,INSTALADO,GAMMA - SIM,DEMO,DEMO,GAMMA DEMO
11049,ICE008,INSTALADO,GAMMA - SIM,DEMO,DEMO,GAMMA DEMO
11050,ICE009,INSTALADO,GAMMA - SIM,DEMO,DEMO,GAMMA DEMO


## Actualización del historico de mensajes en gamma

In [4]:
anomalias = 'Select * From anomaliasVMS'
cnxn = tm.db_connection('resources/cnxn.txt', 'ODBC Driver 17 for SQL Server')
raw_file = pd.read_feather('files/historico_gamma.feather')
last_id = raw_file.iloc[-1]['id']
raw_query = "SELECT id, mensaje, identificador, time, rssi, seq_number, ack, convert(datetime, fecha at time zone 'UTC' at time zone 'Central Standard Time (Mexico)') fecha FROM his_rowDataGamma  WHERE id > {}".format(last_id)
raw_gamma = pd.read_sql(raw_query, cnxn)
raw_gamma = raw_file.append(raw_gamma)
raw_gamma = raw_gamma.reset_index()
raw_gamma.drop(columns='index', inplace=True)
raw_gamma.to_feather('files/historico_gamma.feather')

Driver=ODBC Driver 17 for SQL Server;SERVER=nahualac.database.windows.net,1433;DATABASE=nahualac;UID=Telematica;PWD=Telematic@2021


C:\Users\Cantilever\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\4270567412.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_gamma = raw_file.append(raw_gamma)


In [5]:
raw_gamma
data2 = raw_gamma
historico_gammas = raw_gamma[['identificador','fecha','mensaje']]

## Medidores para cada gamma

In [6]:
#Obtención de medidores para cada gamma

server = 'nahualac.database.windows.net,1433' 
database = 'nahualac' 
username = 'Telematica' 
password = 'Telematic@2021' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
query_meds = '''SELECT
        d.identificador dispositivo, tm.nombre modelo, m.identificador medidor, m.posicion posicion
FROM
        tbl_dispositivo d
INNER JOIN
            tbl_gamma gg
                ON d.id = gg.dispositivoId
INNER JOIN
            tbl_medidor m
                ON gg.id = m.gammaId
INNER JOIN
            tbl_modelo tm on d.modeloId = tm.id
ORDER BY d.id, m.posicion;'''

with warnings.catch_warnings(): 
    warnings.simplefilter('ignore', UserWarning) 
    gamma_meds = pd.read_sql(query_meds, cnxn)
    
gamma_meds

,dispositivo,modelo,medidor,posicion
0,444464,SIM,86T19M,1
1,444464,SIM,86T37M,2
2,444464,SIM,79T02M,3
3,444464,SIM,86T30M,4
4,444464,SIM,86T27M,5
...,...,...,...,...
140042,ICE010,SIM,MED015T49N,7
140043,ICE010,SIM,MED076X7R5,8
140044,ICE010,SIM,MED015T44N,10
140045,ICE010,SIM,MED015T50N,11


# Análisis descriptivo de mensajes en gamma 

In [7]:
historico_gammas['fecha_registro'] = pd.to_datetime(historico_gammas['fecha']).dt.date
historico_gammas['fecha_analisis'] = dt.date.today()
historico_gammas['fecha_analisis'] = pd.to_datetime(historico_gammas['fecha_analisis'])
historico_gammas['fecha'] = pd.to_datetime(historico_gammas['fecha'])

test = historico_gammas
alterno = test
alterno = alterno.drop_duplicates(subset="identificador", keep="last")
alterno = alterno[["identificador","fecha"]]
alterno['fecha_analisis'] = dt.date.today()
alterno = alterno.reset_index(drop=True)
alterno['fecha_registro'] = pd.to_datetime(alterno['fecha']).dt.date

alterno["diff_dias"] = (alterno["fecha_registro"] - alterno["fecha_analisis"])

for i in range (0, len(alterno)):
    alterno["diff_dias"][i] = alterno["diff_dias"][i].days
    
alterno = alterno.drop(['fecha','fecha_analisis','fecha_registro'], axis=1)
alterno.diff_dias = alterno.diff_dias*(-1)
alterno = alterno.rename({"identificador":"dispositivo"},axis=1)

gamma_com_dias = alterno.merge(data, how='inner', on = ['dispositivo'])
gamma_com_dias = gamma_com_dias.rename({"diff_dias":"dias_sin_com"},axis=1)

#Calculo de intervalos de tiempo de comunicacion

sim = gamma_com_dias[gamma_com_dias.modelo=="GAMMA - SIM"]

german = gamma_com_dias[gamma_com_dias.modelo=="GAMMA - GERMAN"]

C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\2800353870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historico_gammas['fecha_registro'] = pd.to_datetime(historico_gammas['fecha']).dt.date
C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\2800353870.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historico_gammas['fecha_analisis'] = dt.date.today()
C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\2800353870.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

# Para calcular la cantidad de mensajes de los ultimos dos meses para GS y GG

In [8]:
gammas_german = historico_gammas[historico_gammas.mensaje.str.startswith("26")]
gammas_sim = historico_gammas[historico_gammas.mensaje.str.startswith("86")]

#Separación entre comando de corte 50 y 55 para german
gammas_german_corte_50 = gammas_german[gammas_german.mensaje.str.startswith("2650")]
gammas_german_corte_55 = gammas_german[gammas_german.mensaje.str.startswith("2655")]
#Solo se usaron comandos 55, los 50 están vacíos
gammas_german_corte_55 = gammas_german_corte_55.reset_index()
gammas_german_corte_55 = gammas_german_corte_55.drop(['index'], axis=1)

#Separación entre comando de reconección 60 y 65 para german
gammas_german_reco_60 = gammas_german[gammas_german.mensaje.str.startswith("2660")]
gammas_german_reco_65 = gammas_german[gammas_german.mensaje.str.startswith("2665")]

#Solo se usaron comandos 65, los 60 están vacíos
gammas_german_reco_65 = gammas_german_reco_65.reset_index()
gammas_german_reco_65 = gammas_german_reco_65.drop(['index'], axis=1)

#Separación entre comando de corte 50 y 55 para sim
gammas_sim_corte_50 = gammas_sim[gammas_sim.mensaje.str.startswith("8650")]
gammas_sim_corte_55 = gammas_sim[gammas_sim.mensaje.str.startswith("8655")]

#Solo se usaron comandos 55, los 50 están vacíos
gammas_sim_corte_55 = gammas_sim_corte_55.reset_index()
gammas_sim_corte_55 = gammas_sim_corte_55.drop(['index'], axis=1)

#Separación entre comando de corte 50 y 55 para sim
gammas_sim_reco_60 = gammas_sim[gammas_sim.mensaje.str.startswith("8660")]
gammas_sim_reco_65 = gammas_sim[gammas_sim.mensaje.str.startswith("8665")]

#Solo se usaron comandos 65, los 60 están vacíos
gammas_sim_reco_65 = gammas_sim_reco_65.reset_index()
gammas_sim_reco_65 = gammas_sim_reco_65.drop(['index'], axis=1)

# Eliminar duplicados
gammas_german_corte_55 = gammas_german_corte_55.drop_duplicates(subset=['mensaje','fecha_registro','identificador'])
gammas_german_reco_65 = gammas_german_reco_65.drop_duplicates(subset=['mensaje','fecha_registro','identificador'])
gammas_sim_corte_55 = gammas_sim_corte_55.drop_duplicates(subset=['mensaje','fecha_registro','identificador'])
gammas_sim_reco_65 = gammas_sim_reco_65.drop_duplicates(subset=['mensaje','fecha_registro','identificador'])

print(len(gammas_german_corte_55))
print(len(gammas_german_reco_65))
print(len(gammas_sim_corte_55))
print(len(gammas_sim_reco_65))

11941
29817
3658
3763


# Parsear mensaje en 3 bytes, pasar a binario y hacer una suma acumulada del cmd a los medidores, y los respectivos medidores

In [9]:
gammas_german_corte_55 = gammas_german_corte_55.reset_index()
gammas_german_corte_55.drop(columns='index', inplace=True)
gammas_german_corte_55['medidores'] = gammas_german_corte_55['mensaje'].apply(lambda x: x[4:10])

gammas_german_reco_65 = gammas_german_reco_65.reset_index()
gammas_german_reco_65.drop(columns='index', inplace=True)
gammas_german_reco_65['medidores'] = gammas_german_reco_65['mensaje'].apply(lambda x: x[4:10])

gammas_sim_corte_55 = gammas_sim_corte_55.reset_index()
gammas_sim_corte_55.drop(columns='index', inplace=True)
gammas_sim_corte_55['medidores'] = gammas_sim_corte_55['mensaje'].apply(lambda x: x[4:10])

gammas_sim_reco_65 = gammas_sim_reco_65.reset_index()
gammas_sim_reco_65.drop(columns='index', inplace=True)
gammas_sim_reco_65['medidores'] = gammas_sim_reco_65['mensaje'].apply(lambda x: x[4:10])

In [10]:
#Para gg cmd 55
gammas_german_corte_55['Byte_3'] = gammas_german_corte_55['medidores'].apply(lambda x: x[0:2])
gammas_german_corte_55['Byte_2'] = gammas_german_corte_55['medidores'].apply(lambda x: x[2:4])
gammas_german_corte_55['Byte_1'] = gammas_german_corte_55['medidores'].apply(lambda x: x[4:6])
gammas_german_corte_55['Meds_Acum'] = None
gammas_german_corte_55['Meds_Acum'] = gammas_german_corte_55['Byte_1'].apply(lambda x: format(int(x, 16), "b")).str.count('1') + gammas_german_corte_55['Byte_2'].apply(lambda x: format(int(x, 16), "b")).str.count('1') + gammas_german_corte_55['Byte_3'].apply(lambda x: format(int(x, 16), "b")).str.count('1')

#Para gg cmd 65
gammas_german_reco_65['Byte_3'] = gammas_german_reco_65['medidores'].apply(lambda x: x[0:2])
gammas_german_reco_65['Byte_2'] = gammas_german_reco_65['medidores'].apply(lambda x: x[2:4])
gammas_german_reco_65['Byte_1'] = gammas_german_reco_65['medidores'].apply(lambda x: x[4:6])
gammas_german_reco_65['Meds_Acum'] = None
gammas_german_reco_65['Meds_Acum'] = gammas_german_reco_65['Byte_1'].apply(lambda x: format(int(x, 16), "b")).str.count('1') + gammas_german_reco_65['Byte_2'].apply(lambda x: format(int(x, 16), "b")).str.count('1') + gammas_german_reco_65['Byte_3'].apply(lambda x: format(int(x, 16), "b")).str.count('1')

#Para gs cmd 55
gammas_sim_corte_55['Byte_3'] = gammas_sim_corte_55['medidores'].apply(lambda x: x[0:2])
gammas_sim_corte_55['Byte_2'] = gammas_sim_corte_55['medidores'].apply(lambda x: x[2:4])
gammas_sim_corte_55['Byte_1'] = gammas_sim_corte_55['medidores'].apply(lambda x: x[4:6])
gammas_sim_corte_55['Meds_Acum'] = None
gammas_sim_corte_55['Meds_Acum'] = gammas_sim_corte_55['Byte_1'].apply(lambda x: format(int(x, 16), "b")).str.count('1') + gammas_sim_corte_55['Byte_2'].apply(lambda x: format(int(x, 16), "b")).str.count('1') + gammas_sim_corte_55['Byte_3'].apply(lambda x: format(int(x, 16), "b")).str.count('1')

#Para gs cmd 65
gammas_sim_reco_65['Byte_3'] = gammas_sim_reco_65['medidores'].apply(lambda x: x[0:2])
gammas_sim_reco_65['Byte_2'] = gammas_sim_reco_65['medidores'].apply(lambda x: x[2:4])
gammas_sim_reco_65['Byte_1'] = gammas_sim_reco_65['medidores'].apply(lambda x: x[4:6])
gammas_sim_reco_65['Meds_Acum'] = None
gammas_sim_reco_65['Meds_Acum'] = gammas_sim_reco_65['Byte_1'].apply(lambda x: format(int(x, 16), "b")).str.count('1') + gammas_sim_reco_65['Byte_2'].apply(lambda x: format(int(x, 16), "b")).str.count('1') + gammas_sim_reco_65['Byte_3'].apply(lambda x: format(int(x, 16), "b")).str.count('1')

## Para obtener a que medidores va cada mensaje

In [11]:
import re

### Obtenemos una copia de identificacion de zona para cada gamma, para su posterior cruce

In [12]:
data_copy = data
data_copy.rename(columns={'dispositivo':'identificador'},inplace=True)

#### Para gg cmd 55

In [13]:
#Cruce con data_copy para tener registro de que zona fue donde se ejecutó el mensaje
gammas_german_corte_55 = gammas_german_corte_55.merge(data_copy, how='inner', on = ['identificador'])

In [14]:
#Analisis 
gammas_german_corte_55['Byte_3_medidores'] = gammas_german_corte_55['Byte_3']
gammas_german_corte_55['Byte_2_medidores'] = gammas_german_corte_55['Byte_2']
gammas_german_corte_55['Byte_1_medidores'] = gammas_german_corte_55['Byte_1']

gammas_german_corte_55['Byte_3_medidores'] = gammas_german_corte_55['Byte_3_medidores'].apply(lambda x: format(int(x, 16), "b"))
gammas_german_corte_55['Byte_2_medidores'] = gammas_german_corte_55['Byte_2_medidores'].apply(lambda x: format(int(x, 16), "b"))
gammas_german_corte_55['Byte_1_medidores'] = gammas_german_corte_55['Byte_1_medidores'].apply(lambda x: format(int(x, 16), "b"))

gammas_german_corte_55['Byte_3_medidores'] = gammas_german_corte_55['Byte_3_medidores'].apply(lambda x: x[::-1])
gammas_german_corte_55['Byte_2_medidores'] = gammas_german_corte_55['Byte_2_medidores'].apply(lambda x: x[::-1])
gammas_german_corte_55['Byte_1_medidores'] = gammas_german_corte_55['Byte_1_medidores'].apply(lambda x: x[::-1])

gammas_german_corte_55['Byte_3_medidores'] = gammas_german_corte_55['Byte_3_medidores'].apply(lambda x: list(((i.start() + 1)+16 for i in re.finditer('1', x))))
gammas_german_corte_55['Byte_2_medidores'] = gammas_german_corte_55['Byte_2_medidores'].apply(lambda x: list(((i.start() + 1)+8 for i in re.finditer('1', x))))
gammas_german_corte_55['Byte_1_medidores'] = gammas_german_corte_55['Byte_1_medidores'].apply(lambda x: list(((i.start() + 1) for i in re.finditer('1', x))))
gammas_german_corte_55['posicion_medidores'] = gammas_german_corte_55['Byte_3_medidores'] + gammas_german_corte_55['Byte_2_medidores'] + gammas_german_corte_55['Byte_1_medidores']

gammas_german_corte_55 = gammas_german_corte_55.drop(['fecha_analisis','medidores','Byte_3_medidores','Byte_2_medidores','Byte_1_medidores'], axis=1)

gammas_german_corte_55

,identificador,fecha,mensaje,fecha_registro,Byte_3,Byte_2,Byte_1,Meds_Acum,estatus,modelo,division,zona,proyecto,posicion_medidores
0,48C4B3,2022-02-08 16:07:53.283,2655080000ffffffffffff83,2022-02-08,08,00,00,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,TOLUCA,GAMMA NACIONALES,[20]
1,48C4B3,2022-02-09 00:02:21.437,2655080000ffffffffffff83,2022-02-09,08,00,00,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,TOLUCA,GAMMA NACIONALES,[20]
2,48C4B3,2022-04-13 12:01:39.297,2655080000ffffffffffff83,2022-04-13,08,00,00,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,TOLUCA,GAMMA NACIONALES,[20]
3,48C4B3,2022-04-14 00:05:50.987,2655080000ffffffffffff83,2022-04-14,08,00,00,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,TOLUCA,GAMMA NACIONALES,[20]
4,48C4B3,2022-04-15 00:14:45.707,2655080000ffffffffffff83,2022-04-15,08,00,00,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,TOLUCA,GAMMA NACIONALES,[20]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11907,7F8907,2022-08-02 12:55:18.330,2655030000ffffffffffff88,2022-08-02,03,00,00,2,INSTALADO,GAMMA - GERMAN,Valle de México Sur,LOMAS,GAMMA NACIONALES,"[17, 18]"
11908,804742,2022-08-02 15:01:16.853,2655000e00ffffffffffff7d,2022-08-02,00,0e,00,3,INSTALADO,GAMMA - GERMAN,Valle de México Sur,LOMAS,GAMMA NACIONALES,"[10, 11, 12]"
11909,456377,2022-08-02 17:14:19.950,2655000400ffffffffffff87,2022-08-02,00,04,00,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,LOMAS,GAMMA IOT,[11]
11910,4239B3,2022-08-02 17:16:01.373,2655080000ffffffffffff83,2022-08-02,08,00,00,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,UNIVERSIDAD,GAMMA NACIONALES,[20]


#### Para gg cmd 65

In [15]:
#Cruce con data_copy para tener registro de que zona fue donde se ejecutó el mensaje
gammas_german_reco_65 = gammas_german_reco_65.merge(data_copy, how='inner', on = ['identificador'])

#Analisis
gammas_german_reco_65['Byte_3_medidores'] = gammas_german_reco_65['Byte_3']
gammas_german_reco_65['Byte_2_medidores'] = gammas_german_reco_65['Byte_2']
gammas_german_reco_65['Byte_1_medidores'] = gammas_german_reco_65['Byte_1']

gammas_german_reco_65['Byte_3_medidores'] = gammas_german_reco_65['Byte_3_medidores'].apply(lambda x: format(int(x, 16), "b"))
gammas_german_reco_65['Byte_2_medidores'] = gammas_german_reco_65['Byte_2_medidores'].apply(lambda x: format(int(x, 16), "b"))
gammas_german_reco_65['Byte_1_medidores'] = gammas_german_reco_65['Byte_1_medidores'].apply(lambda x: format(int(x, 16), "b"))

gammas_german_reco_65['Byte_3_medidores'] = gammas_german_reco_65['Byte_3_medidores'].apply(lambda x: x[::-1])
gammas_german_reco_65['Byte_2_medidores'] = gammas_german_reco_65['Byte_2_medidores'].apply(lambda x: x[::-1])
gammas_german_reco_65['Byte_1_medidores'] = gammas_german_reco_65['Byte_1_medidores'].apply(lambda x: x[::-1])

gammas_german_reco_65['Byte_3_medidores'] = gammas_german_reco_65['Byte_3_medidores'].apply(lambda x: list(((i.start() + 1)+16 for i in re.finditer('1', x))))
gammas_german_reco_65['Byte_2_medidores'] = gammas_german_reco_65['Byte_2_medidores'].apply(lambda x: list(((i.start() + 1)+8 for i in re.finditer('1', x))))
gammas_german_reco_65['Byte_1_medidores'] = gammas_german_reco_65['Byte_1_medidores'].apply(lambda x: list(((i.start() + 1) for i in re.finditer('1', x))))
gammas_german_reco_65['posicion_medidores'] = gammas_german_reco_65['Byte_3_medidores'] + gammas_german_reco_65['Byte_2_medidores'] + gammas_german_reco_65['Byte_1_medidores']

gammas_german_reco_65 = gammas_german_reco_65.drop(['fecha_analisis','medidores','Byte_3_medidores','Byte_2_medidores','Byte_1_medidores'], axis=1)

gammas_german_reco_65

,identificador,fecha,mensaje,fecha_registro,Byte_3,Byte_2,Byte_1,Meds_Acum,estatus,modelo,division,zona,proyecto,posicion_medidores
0,494551,2022-02-08 13:58:11.587,2665000000ffffffffffff7b,2022-02-08,00,00,00,0,INSTALADO,GAMMA - GERMAN,Valle de México Sur,COAPA,GAMMA IOT,[]
1,494551,2022-02-09 00:01:42.170,2665000000ffffffffffff7b,2022-02-09,00,00,00,0,INSTALADO,GAMMA - GERMAN,Valle de México Sur,COAPA,GAMMA IOT,[]
2,494551,2022-04-12 18:53:37.613,2665000000ffffffffffff7b,2022-04-12,00,00,00,0,INSTALADO,GAMMA - GERMAN,Valle de México Sur,COAPA,GAMMA IOT,[]
3,494551,2022-04-13 00:34:41.567,2665000000ffffffffffff7b,2022-04-13,00,00,00,0,INSTALADO,GAMMA - GERMAN,Valle de México Sur,COAPA,GAMMA IOT,[]
4,494551,2022-04-14 00:44:09.497,2665000000ffffffffffff7b,2022-04-14,00,00,00,0,INSTALADO,GAMMA - GERMAN,Valle de México Sur,COAPA,GAMMA IOT,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29693,81C463,2022-08-02 21:09:59.820,2665000020ffffffffffff5b,2022-08-02,00,00,20,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,LOMAS,GAMMA NACIONALES,[6]
29694,4239B3,2022-08-02 22:13:08.367,2665080000ffffffffffff73,2022-08-02,08,00,00,1,INSTALADO,GAMMA - GERMAN,Valle de México Sur,UNIVERSIDAD,GAMMA NACIONALES,[20]
29695,447105,2022-08-03 02:05:07.023,2665000e00ffffffffffff6d,2022-08-03,00,0e,00,3,INSTALADO,GAMMA - GERMAN,Valle de México Sur,LOMAS,GAMMA NACIONALES AMP,"[10, 11, 12]"
29696,804B74,2022-08-03 04:54:33.307,26651fffffffffffffffff5e,2022-08-03,1f,ff,ff,21,INSTALADO,GAMMA - GERMAN,Valle de México Sur,LOMAS,GAMMA NACIONALES,"[17, 18, 19, 20, 21, 9, 10, 11, 12, 13, 14, 15..."


#### Para gs cmd 55

In [16]:
#Cruce con data_copy para tener registro de que zona fue donde se ejecutó el mensaje
gammas_sim_corte_55 = gammas_sim_corte_55.merge(data_copy, how='inner', on = ['identificador'])

#Analisis

gammas_sim_corte_55['Byte_3_medidores'] = gammas_sim_corte_55['Byte_3']
gammas_sim_corte_55['Byte_2_medidores'] = gammas_sim_corte_55['Byte_2']
gammas_sim_corte_55['Byte_1_medidores'] = gammas_sim_corte_55['Byte_1']

gammas_sim_corte_55['Byte_3_medidores'] = gammas_sim_corte_55['Byte_3_medidores'].apply(lambda x: format(int(x, 16), "b"))
gammas_sim_corte_55['Byte_2_medidores'] = gammas_sim_corte_55['Byte_2_medidores'].apply(lambda x: format(int(x, 16), "b"))
gammas_sim_corte_55['Byte_1_medidores'] = gammas_sim_corte_55['Byte_1_medidores'].apply(lambda x: format(int(x, 16), "b"))

gammas_sim_corte_55['Byte_3_medidores'] = gammas_sim_corte_55['Byte_3_medidores'].apply(lambda x: x[::-1])
gammas_sim_corte_55['Byte_2_medidores'] = gammas_sim_corte_55['Byte_2_medidores'].apply(lambda x: x[::-1])
gammas_sim_corte_55['Byte_1_medidores'] = gammas_sim_corte_55['Byte_1_medidores'].apply(lambda x: x[::-1])

gammas_sim_corte_55['Byte_3_medidores'] = gammas_sim_corte_55['Byte_3_medidores'].apply(lambda x: list(((i.start() + 1)+16 for i in re.finditer('1', x))))
gammas_sim_corte_55['Byte_2_medidores'] = gammas_sim_corte_55['Byte_2_medidores'].apply(lambda x: list(((i.start() + 1)+8 for i in re.finditer('1', x))))
gammas_sim_corte_55['Byte_1_medidores'] = gammas_sim_corte_55['Byte_1_medidores'].apply(lambda x: list(((i.start() + 1) for i in re.finditer('1', x))))
gammas_sim_corte_55['posicion_medidores'] = gammas_sim_corte_55['Byte_3_medidores'] + gammas_sim_corte_55['Byte_2_medidores'] + gammas_sim_corte_55['Byte_1_medidores']

gammas_sim_corte_55 = gammas_sim_corte_55.drop(['fecha_analisis','medidores','Byte_3_medidores','Byte_2_medidores','Byte_1_medidores'], axis=1)

gammas_sim_corte_55

,identificador,fecha,mensaje,fecha_registro,Byte_3,Byte_2,Byte_1,Meds_Acum,estatus,modelo,division,zona,proyecto,posicion_medidores
0,448440,2021-10-08 09:44:15.903,8655000130fffffffffffffa,2021-10-08,00,01,30,3,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3,"[9, 5, 6]"
1,448440,2021-12-08 11:28:57.877,8655000010ffffffffffff1b,2021-12-08,00,00,10,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3,[5]
2,448440,2022-02-08 10:08:38.763,8655000030fffffffffffffb,2022-02-08,00,00,30,2,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3,"[5, 6]"
3,426637,2021-10-08 09:44:52.323,8655000100ffffffffffff2a,2021-10-08,00,01,00,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 1,GAMMA ACA 1,[9]
4,426637,2022-02-08 10:05:23.040,8655000100ffffffffffff2a,2022-02-08,00,01,00,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 1,GAMMA ACA 1,[9]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3643,81C415,2022-07-13 16:49:38.603,8655000001ffffffffffff2a,2022-07-13,00,00,01,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3 AMP,[1]
3644,44BB60,2022-07-13 16:57:02.373,8655000000ffffffffffff2b,2022-07-13,00,00,00,0,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3,[]
3645,4482A0,2022-07-18 15:24:10.237,8655000010ffffffffffff1b,2022-07-18,00,00,10,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3,[5]
3646,440B6E,2022-07-20 13:24:03.840,8655000100ffffffffffff2a,2022-07-20,00,01,00,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO,GAMMA ACA 4,[9]


#### Para gs cmd 65

In [17]:
#Cruce con data_copy para tener registro de que zona fue donde se ejecutó el mensaje
gammas_sim_reco_65 = gammas_sim_reco_65.merge(data_copy, how='inner', on = ['identificador'])

#Analisis

gammas_sim_reco_65['Byte_3_medidores'] = gammas_sim_reco_65['Byte_3']
gammas_sim_reco_65['Byte_2_medidores'] = gammas_sim_reco_65['Byte_2']
gammas_sim_reco_65['Byte_1_medidores'] = gammas_sim_reco_65['Byte_1']

gammas_sim_reco_65['Byte_3_medidores'] = gammas_sim_reco_65['Byte_3_medidores'].apply(lambda x: format(int(x, 16), "b"))
gammas_sim_reco_65['Byte_2_medidores'] = gammas_sim_reco_65['Byte_2_medidores'].apply(lambda x: format(int(x, 16), "b"))
gammas_sim_reco_65['Byte_1_medidores'] = gammas_sim_reco_65['Byte_1_medidores'].apply(lambda x: format(int(x, 16), "b"))

gammas_sim_reco_65['Byte_3_medidores'] = gammas_sim_reco_65['Byte_3_medidores'].apply(lambda x: x[::-1])
gammas_sim_reco_65['Byte_2_medidores'] = gammas_sim_reco_65['Byte_2_medidores'].apply(lambda x: x[::-1])
gammas_sim_reco_65['Byte_1_medidores'] = gammas_sim_reco_65['Byte_1_medidores'].apply(lambda x: x[::-1])

gammas_sim_reco_65['Byte_3_medidores'] = gammas_sim_reco_65['Byte_3_medidores'].apply(lambda x: list(((i.start() + 1)+16 for i in re.finditer('1', x))))
gammas_sim_reco_65['Byte_2_medidores'] = gammas_sim_reco_65['Byte_2_medidores'].apply(lambda x: list(((i.start() + 1)+8 for i in re.finditer('1', x))))
gammas_sim_reco_65['Byte_1_medidores'] = gammas_sim_reco_65['Byte_1_medidores'].apply(lambda x: list(((i.start() + 1) for i in re.finditer('1', x))))
gammas_sim_reco_65['posicion_medidores'] = gammas_sim_reco_65['Byte_3_medidores'] + gammas_sim_reco_65['Byte_2_medidores'] + gammas_sim_reco_65['Byte_1_medidores']

gammas_sim_reco_65 = gammas_sim_reco_65.drop(['fecha_analisis','medidores','Byte_3_medidores','Byte_2_medidores','Byte_1_medidores'], axis=1)

gammas_sim_reco_65

,identificador,fecha,mensaje,fecha_registro,Byte_3,Byte_2,Byte_1,Meds_Acum,estatus,modelo,division,zona,proyecto,posicion_medidores
0,423674,2021-10-07 17:46:19.373,8665000400ffffffffffff17,2021-10-07,00,04,00,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 1,GAMMA ACA 1,[11]
1,423674,2021-10-13 14:53:15.323,8665000200ffffffffffff19,2021-10-13,00,02,00,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 1,GAMMA ACA 1,[10]
2,423674,2022-01-14 14:00:26.563,8665000004ffffffffffff17,2022-01-14,00,00,04,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 1,GAMMA ACA 1,[3]
3,423674,2022-01-17 12:00:20.223,8665000004ffffffffffff17,2022-01-17,00,00,04,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 1,GAMMA ACA 1,[3]
4,497F44,2021-10-07 18:16:08.920,8665000002ffffffffffff19,2021-10-07,00,00,02,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3,[2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,8046E1,2022-07-27 20:54:14.533,8665000080ffffffffffff9b,2022-07-27,00,00,80,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3 AMP,[8]
3748,45083F,2022-07-28 08:32:41.580,8665000004ffffffffffff17,2022-07-28,00,00,04,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3,[3]
3749,81B8AD,2022-07-28 16:31:02.623,8665000002ffffffffffff19,2022-07-28,00,00,02,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3 AMP,[2]
3750,49072A,2022-08-02 10:51:28.903,8665000080ffffffffffff9b,2022-08-02,00,00,80,1,INSTALADO,GAMMA - SIM,Centro Sur,ACAPULCO 3,GAMMA ACA 3,[8]


# Limpieza y tratamiento de datos, segmentación y clasificación por mes

In [18]:
#Renombrar variables de datos
gg_cmd_55 = gammas_german_corte_55
gg_cmd_55["month"] = gg_cmd_55.fecha.dt.month
gg_cmd_55["year"] = gg_cmd_55.fecha.dt.year

gg_cmd_65 = gammas_german_reco_65
gg_cmd_65["month"] = gg_cmd_65.fecha.dt.month
gg_cmd_65["year"] = gg_cmd_65.fecha.dt.year

gs_cmd_55 = gammas_sim_corte_55
gs_cmd_55["month"] = gs_cmd_55.fecha.dt.month
gs_cmd_55["year"] = gs_cmd_55.fecha.dt.year

gs_cmd_65 = gammas_sim_reco_65
gs_cmd_65["month"] = gs_cmd_65.fecha.dt.month
gs_cmd_65["year"] = gs_cmd_65.fecha.dt.year

#Identificamos los indices de los datos demo
demo_gg_55 = list(np.where(gg_cmd_55.zona == "DEMO"))
ermita_gg_55 = list(np.where(gg_cmd_55.zona == "ERMITA"))

demo_gg_65 = list(np.where(gg_cmd_65.zona == "DEMO"))
ermita_gg_65 = list(np.where(gg_cmd_65.zona == "ERMITA"))

#Eliminamos del df los datos de no interes
gg_cmd_55.drop(gg_cmd_55.index[demo_gg_55],0,inplace=True)
gg_cmd_55.drop(gg_cmd_55.index[ermita_gg_55],0,inplace=True)

gg_cmd_65.drop(gg_cmd_65.index[demo_gg_65],0,inplace=True)
gg_cmd_65.drop(gg_cmd_65.index[ermita_gg_65],0,inplace=True)

#Reseteamos indices, borramos columna de indices
gg_cmd_55 = gg_cmd_55.reset_index()
gg_cmd_65 = gg_cmd_65.reset_index()

gg_cmd_55 = gg_cmd_55.drop(["index"], axis=1)
gg_cmd_65 = gg_cmd_65.drop(["index"], axis=1)

zonas_gg = gg_cmd_55.zona.unique()
zonas_gs = gs_cmd_55.zona.unique()

C:\Users\Cantilever\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexes\base.py:5055: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\1454603871.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  gg_cmd_55.drop(gg_cmd_55.index[demo_gg_55],0,inplace=True)
C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\1454603871.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  gg_cmd_55.drop(gg_cmd_55.index[ermita_gg_55],0,inplace=True)
C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\145460

# Extracción de data (última actualización en reunión)

## Función para determinar cantidad de dias según el numero de mes

In [19]:
def es_bisiesto(year: int) -> bool:
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)


def obtener_dias_del_mes(mes: int, year: int) -> int:
    # Abril, junio, septiembre y noviembre tienen 30
    if mes in [4, 6, 9, 11]:
        return 30
    # Febrero depende de si es o no bisiesto
    if mes == 2:
        if es_bisiesto(year):
            return 29
        else:
            return 28
    else:
        # En caso contrario, tiene 31 días
        return 31

## Eliminar a cada df registros previos al Abril de 2022, así como también los ceros en Meds_Acum

In [20]:
#Filtración de datos pasados, sin interes
gg_cmd_55 = gg_cmd_55[(gg_cmd_55["month"]>=4) & (gg_cmd_55["year"]==2022)]
gg_cmd_65 = gg_cmd_65[(gg_cmd_65["month"]>=4) & (gg_cmd_65["year"]==2022)]
gs_cmd_55 = gs_cmd_55[(gs_cmd_55["month"]>=4) & (gs_cmd_55["year"]==2022)]
gs_cmd_65 = gs_cmd_65[(gs_cmd_65["month"]>=4) & (gs_cmd_65["year"]==2022)]

#Reset para los indices
gg_cmd_55.drop(gg_cmd_55.loc[gg_cmd_55['Meds_Acum']==0].index, inplace=True)
gg_cmd_55 = gg_cmd_55.reset_index()
gg_cmd_55 = gg_cmd_55.drop(["index"], axis=1)

gg_cmd_65.drop(gg_cmd_65.loc[gg_cmd_65['Meds_Acum']==0].index, inplace=True)
gg_cmd_65 = gg_cmd_65.reset_index()
gg_cmd_65 = gg_cmd_65.drop(["index"], axis=1)

gs_cmd_55.drop(gs_cmd_55.loc[gs_cmd_55['Meds_Acum']==0].index, inplace=True)
gs_cmd_55 = gs_cmd_55.reset_index()
gs_cmd_55 = gs_cmd_55.drop(["index"], axis=1)

gs_cmd_65.drop(gs_cmd_65.loc[gs_cmd_65['Meds_Acum']==0].index, inplace=True)
gs_cmd_65 = gs_cmd_65.reset_index()
gs_cmd_65 = gs_cmd_65.drop(["index"], axis=1)

C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\4260645286.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gs_cmd_55.drop(gs_cmd_55.loc[gs_cmd_55['Meds_Acum']==0].index, inplace=True)
C:\Users\Cantilever\AppData\Local\Temp\ipykernel_8860\4260645286.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gs_cmd_65.drop(gs_cmd_65.loc[gs_cmd_65['Meds_Acum']==0].index, inplace=True)


# Nuevo agrupamiento para las gammas
# DataFrames agrupados por mes (ya no por zona)

## Empezamos el análisis con gamma german

### Zonas gg

In [21]:
zonas_gg = list(zonas_gg)
zonas_gg

['TOLUCA', 'UNIVERSIDAD', 'TENANGO', 'COAPA', 'LOMAS']

## Nuevo algoritmo de segmentación de datos

In [22]:
#Creamos un df contenedor que fungira como tabla resumen historica para gg cmd 55
gg_cmd_55_data = pd.DataFrame()
gg_cmd_55_data = gg_cmd_55_data.assign(year=None,month=None,divs=None,msgs=None,zona=None)

#Obtenemos los valores únicos para meses y año, para el df antes descrito
gg_55_months = list(gg_cmd_55["month"].unique())
gg_55_months = sorted(gg_55_months)
gg_55_year = list(gg_cmd_55["year"].unique())

#Nueva formula
# 3(intervalos de tiempo)*número de divisiones(dependerá de modelo de gamma)
rows = 3*len(zonas_gg)

#Para crear un df vacío con los renglones necesarios, poniendo únicamente el año en cuestión
for i in range (0, rows):
    gg_cmd_55_data = pd.concat([gg_cmd_55_data, pd.DataFrame({'year':[gg_55_year[-1]], 'month':[None], 
                                                              'divs':[None], 'msgs':[None], 'zona':[None]})])
    
#Reseteamos los indices    
gg_cmd_55_data = gg_cmd_55_data.reset_index()
gg_cmd_55_data = gg_cmd_55_data.drop(["index"], axis= 1)

#Se realiza un for para ir llenando el df resumen para ROI, en meses, zonas e intervalos de zonas
contador = 0

for i in range (0,  len(zonas_gg)):
    gg_cmd_55_data.zona[contador:contador+3] = zonas_gg[i]
    contador = contador + 3
    
# Se realizara el vaciado de meses en una lista contenedora

gg_55_div_meses = []
for i in range (0, len(gg_55_months)):
    aux = gg_cmd_55_data.copy()
    aux.month = gg_55_months[i]
    gg_55_div_meses.append(aux)
    
# Rellenar la columna de divs, los intervalos de tiempo de acuerdo al mes en cuestion    

for i in range (0, len(gg_55_div_meses)):
    contador = 0 
    for j in range (0, len(zonas_gg)):
        # Primeros dos intervalos en dias
        gg_55_div_meses[i].at[(0 + (3*j)), 'divs'] = [1, 10]
        gg_55_div_meses[i].at[(1 + (3*j)), 'divs'] = [11, 20]
        #Actualizamos contador, obtenemos el día final de mes
        contador = contador + 3 
        final_day = obtener_dias_del_mes(gg_55_div_meses[i].month[0],gg_55_year[-1])
        # # Colocamos el último intervalo
        row = contador -1
        gg_55_div_meses[i].at[row, "divs"] = [21, final_day]
        
#Pasamos a tupla la lista de los medidores para eliminar duplicados de cmd y medidores en un determinado periodo
gg_cmd_55.posicion_medidores = gg_cmd_55.posicion_medidores.apply(lambda x: tuple(x))

#Contenedor general del df, para evidencia
gg_55_ROI = pd.DataFrame()

for i in range (0, len(gg_55_months)): # Para recorrer por meses la lista de dataframes
    
    for j in range (0, 3): # El 3, porque son 3 intervalos de tiempo por mes
        
        #Delimitamos fechas, fecha inicial, fecha final, para el filtrado de datos desde gg_cmd_55
        initial_date = dt.date(gg_55_div_meses[i].year[j], gg_55_div_meses[i].month[j], gg_55_div_meses[i].divs[j][0])
        last_date = dt.date(gg_55_div_meses[i].year[j], gg_55_div_meses[i].month[j], gg_55_div_meses[i].divs[j][1])
        #Filtrado del df completo de msgs
        aux_cmd = gg_cmd_55.loc[gg_cmd_55["fecha_registro"].between(initial_date, last_date)]
        aux_cmd = aux_cmd.drop_duplicates(subset=["identificador", "posicion_medidores"],keep="first")
        aux_cmd = aux_cmd.reset_index()
        aux_cmd = aux_cmd.drop(["index"], axis = 1)
        #Concatenamos en un df cada uno de los renglones aux
        gg_55_ROI = pd.concat([gg_55_ROI, aux_cmd])
        
        for k in range (0, len(zonas_gg)):
            
            aux_zonas = pd.DataFrame()
            aux_zonas = aux_cmd[aux_cmd.zona == zonas_gg[k]]
            gg_55_div_meses[i].at[(k + (2*k)) + j, "msgs"] = sum(aux_zonas.Meds_Acum)
            gg_55_div_meses[i].at[(k + (2*k)) + j, "zona"] = zonas_gg[k]
            #gg_cmd_55_zonas[j] = gg_cmd_55_zonas[j].sort_values(['month', 'zona'],ascending=True)

# Análisis con gg cmd 65

In [23]:
#Creamos un df contenedor que fungira como tabla resumen historica para gg cmd 55
gg_cmd_65_data = pd.DataFrame()
gg_cmd_65_data = gg_cmd_65_data.assign(year=None,month=None,divs=None,msgs=None,zona=None)

#Obtenemos los valores únicos para meses y año, para el df antes descrito
gg_65_months = list(gg_cmd_65["month"].unique())
gg_65_months = sorted(gg_65_months)
gg_65_year = list(gg_cmd_65["year"].unique())

#Nueva formula
# 3(intervalos de tiempo)*número de divisiones(dependerá de modelo de gamma)
rows = 3*len(zonas_gg)

#Para crear un df vacío con los renglones necesarios, poniendo únicamente el año en cuestión
for i in range (0, rows):
    gg_cmd_65_data = pd.concat([gg_cmd_65_data, pd.DataFrame({'year':[gg_65_year[-1]], 'month':[None], 
                                                              'divs':[None], 'msgs':[None], 'zona':[None]})])
    
#Reseteamos los indices    
gg_cmd_65_data = gg_cmd_65_data.reset_index()
gg_cmd_65_data = gg_cmd_65_data.drop(["index"], axis= 1)

#Se realiza un for para ir llenando el df resumen para ROI, en meses, zonas e intervalos de zonas
contador = 0

for i in range (0,  len(zonas_gg)):
    gg_cmd_65_data.zona[contador:contador+3] = zonas_gg[i]
    contador = contador + 3
    
# Se realizara el vaciado de meses en una lista contenedora

gg_65_div_meses = []
for i in range (0, len(gg_65_months)):
    aux = gg_cmd_65_data.copy()
    aux.month = gg_65_months[i]
    gg_65_div_meses.append(aux)
    
# Rellenar la columna de divs, los intervalos de tiempo de acuerdo al mes en cuestion    

for i in range (0, len(gg_65_div_meses)):
    contador = 0 
    for j in range (0, len(zonas_gg)):
        # Primeros dos intervalos en dias
        gg_65_div_meses[i].at[(0 + (3*j)), 'divs'] = [1, 10]
        gg_65_div_meses[i].at[(1 + (3*j)), 'divs'] = [11, 20]
        #Actualizamos contador, obtenemos el día final de mes
        contador = contador + 3 
        final_day = obtener_dias_del_mes(gg_65_div_meses[i].month[0],gg_65_year[-1])
        # # Colocamos el último intervalo
        row = contador -1
        gg_65_div_meses[i].at[row, "divs"] = [21, final_day]
        
#Pasamos a tupla la lista de los medidores para eliminar duplicados de cmd y medidores en un determinado periodo
gg_cmd_65.posicion_medidores = gg_cmd_65.posicion_medidores.apply(lambda x: tuple(x))

#Contenedor general del df, para evidencia
gg_65_ROI = pd.DataFrame()

for i in range (0, len(gg_65_months)): # Para recorrer por meses la lista de dataframes
    
    for j in range (0, 3): # El 3, porque son 3 intervalos de tiempo por mes
        
        #Delimitamos fechas, fecha inicial, fecha final, para el filtrado de datos desde gg_cmd_55
        initial_date = dt.date(gg_65_div_meses[i].year[j], gg_65_div_meses[i].month[j], gg_65_div_meses[i].divs[j][0])
        last_date = dt.date(gg_65_div_meses[i].year[j], gg_65_div_meses[i].month[j], gg_65_div_meses[i].divs[j][1])
        #Filtrado del df completo de msgs
        aux_cmd = gg_cmd_65.loc[gg_cmd_65["fecha_registro"].between(initial_date, last_date)]
        aux_cmd = aux_cmd.drop_duplicates(subset=["identificador", "posicion_medidores"],keep="first")
        aux_cmd = aux_cmd.reset_index()
        aux_cmd = aux_cmd.drop(["index"], axis = 1)
        #Concatenamos en un df cada uno de los renglones aux
        gg_65_ROI = pd.concat([gg_65_ROI, aux_cmd])
        
        for k in range (0, len(zonas_gg)):
            
            aux_zonas = pd.DataFrame()
            aux_zonas = aux_cmd[aux_cmd.zona == zonas_gg[k]]
            gg_65_div_meses[i].at[(k + (2*k)) + j, "msgs"] = sum(aux_zonas.Meds_Acum)
            gg_65_div_meses[i].at[(k + (2*k)) + j, "zona"] = zonas_gg[k]
            #gg_cmd_55_zonas[j] = gg_cmd_55_zonas[j].sort_values(['month', 'zona'],ascending=True)

# Análisis con gs cmd 55

## Zonas gs

In [24]:
zonas_gs = list(zonas_gs)
zonas_gs

['ACAPULCO 3', 'ACAPULCO 1', 'ACAPULCO 2', 'ACAPULCO']

In [25]:
#Creamos un df contenedor que fungira como tabla resumen historica para gg cmd 55
gs_cmd_55_data = pd.DataFrame()
gs_cmd_55_data = gs_cmd_55_data.assign(year=None,month=None,divs=None,msgs=None,zona=None)

#Obtenemos los valores únicos para meses y año, para el df antes descrito
gs_55_months = list(gs_cmd_65["month"].unique())
gs_55_months = sorted(gs_55_months)
gs_55_year = list(gs_cmd_55["year"].unique())

#Nueva formula
# 3(intervalos de tiempo)*número de divisiones(dependerá de modelo de gamma)
rows = 3*len(zonas_gs)

#Para crear un df vacío con los renglones necesarios, poniendo únicamente el año en cuestión
for i in range (0, rows):
    gs_cmd_55_data = pd.concat([gs_cmd_55_data, pd.DataFrame({'year':[gs_55_year[-1]], 'month':[None], 
                                                              'divs':[None], 'msgs':[None], 'zona':[None]})])
    
#Reseteamos los indices    
gs_cmd_55_data = gs_cmd_55_data.reset_index()
gs_cmd_55_data = gs_cmd_55_data.drop(["index"], axis= 1)

#Se realiza un for para ir llenando el df resumen para ROI, en meses, zonas e intervalos de zonas
contador = 0

for i in range (0,  len(zonas_gs)):
    gs_cmd_55_data.zona[contador:contador+3] = zonas_gs[i]
    contador = contador + 3
    
# Se realizara el vaciado de meses en una lista contenedora

gs_55_div_meses = []
for i in range (0, len(gs_55_months)):
    aux = gs_cmd_55_data.copy()
    aux.month = gs_55_months[i]
    gs_55_div_meses.append(aux)
    
# Rellenar la columna de divs, los intervalos de tiempo de acuerdo al mes en cuestion    

for i in range (0, len(gs_55_div_meses)):
    contador = 0 
    for j in range (0, len(zonas_gs)):
        # Primeros dos intervalos en dias
        gs_55_div_meses[i].at[(0 + (3*j)), 'divs'] = [1, 10]
        gs_55_div_meses[i].at[(1 + (3*j)), 'divs'] = [11, 20]
        #Actualizamos contador, obtenemos el día final de mes
        contador = contador + 3 
        final_day = obtener_dias_del_mes(gs_55_div_meses[i].month[0],gs_55_year[-1])
        # # Colocamos el último intervalo
        row = contador -1
        gs_55_div_meses[i].at[row, "divs"] = [21, final_day]
        
#Pasamos a tupla la lista de los medidores para eliminar duplicados de cmd y medidores en un determinado periodo
gs_cmd_55.posicion_medidores = gs_cmd_55.posicion_medidores.apply(lambda x: tuple(x))

#Contenedor general del df, para evidencia
gs_55_ROI = pd.DataFrame()

for i in range (0, len(gs_55_months)): # Para recorrer por meses la lista de dataframes
    
    for j in range (0, 3): # El 3, porque son 3 intervalos de tiempo por mes
        
        #Delimitamos fechas, fecha inicial, fecha final, para el filtrado de datos desde gg_cmd_55
        initial_date = dt.date(gs_55_div_meses[i].year[j], gs_55_div_meses[i].month[j], gs_55_div_meses[i].divs[j][0])
        last_date = dt.date(gs_55_div_meses[i].year[j], gs_55_div_meses[i].month[j], gs_55_div_meses[i].divs[j][1])
        #Filtrado del df completo de msgs
        aux_cmd = gs_cmd_55.loc[gs_cmd_55["fecha_registro"].between(initial_date, last_date)]
        aux_cmd = aux_cmd.drop_duplicates(subset=["identificador", "posicion_medidores"],keep="first")
        aux_cmd = aux_cmd.reset_index()
        aux_cmd = aux_cmd.drop(["index"], axis = 1)
        #Concatenamos en un df cada uno de los renglones aux
        gs_55_ROI = pd.concat([gs_55_ROI, aux_cmd])
        
        for k in range (0, len(zonas_gs)):
            
            aux_zonas = pd.DataFrame()
            aux_zonas = aux_cmd[aux_cmd.zona == zonas_gs[k]]
            gs_55_div_meses[i].at[(k + (2*k)) + j, "msgs"] = sum(aux_zonas.Meds_Acum)
            gs_55_div_meses[i].at[(k + (2*k)) + j, "zona"] = zonas_gs[k]
            #gg_cmd_55_zonas[j] = gg_cmd_55_zonas[j].sort_values(['month', 'zona'],ascending=True)

# Análisis con gs cmd 65

In [26]:
#Creamos un df contenedor que fungira como tabla resumen historica para gg cmd 55
gs_cmd_65_data = pd.DataFrame()
gs_cmd_65_data = gs_cmd_65_data.assign(year=None,month=None,divs=None,msgs=None,zona=None)

#Obtenemos los valores únicos para meses y año, para el df antes descrito
gs_65_months = list(gs_cmd_65["month"].unique())
gs_65_months = sorted(gs_65_months)
gs_65_year = list(gs_cmd_65["year"].unique())

#Nueva formula
# 3(intervalos de tiempo)*número de divisiones(dependerá de modelo de gamma)
rows = 3*len(zonas_gs)

#Para crear un df vacío con los renglones necesarios, poniendo únicamente el año en cuestión
for i in range (0, rows):
    gs_cmd_65_data = pd.concat([gs_cmd_65_data, pd.DataFrame({'year':[gs_65_year[-1]], 'month':[None], 
                                                              'divs':[None], 'msgs':[None], 'zona':[None]})])
    
#Reseteamos los indices    
gs_cmd_65_data = gs_cmd_65_data.reset_index()
gs_cmd_65_data = gs_cmd_65_data.drop(["index"], axis= 1)

#Se realiza un for para ir llenando el df resumen para ROI, en meses, zonas e intervalos de zonas
contador = 0

for i in range (0,  len(zonas_gs)):
    gs_cmd_65_data.zona[contador:contador+3] = zonas_gs[i]
    contador = contador + 3
    
# Se realizara el vaciado de meses en una lista contenedora

gs_65_div_meses = []
for i in range (0, len(gs_65_months)):
    aux = gs_cmd_65_data.copy()
    aux.month = gs_65_months[i]
    gs_65_div_meses.append(aux)
    
# Rellenar la columna de divs, los intervalos de tiempo de acuerdo al mes en cuestion    

for i in range (0, len(gs_65_div_meses)):
    contador = 0 
    for j in range (0, len(zonas_gs)):
        # Primeros dos intervalos en dias
        gs_65_div_meses[i].at[(0 + (3*j)), 'divs'] = [1, 10]
        gs_65_div_meses[i].at[(1 + (3*j)), 'divs'] = [11, 20]
        #Actualizamos contador, obtenemos el día final de mes
        contador = contador + 3 
        final_day = obtener_dias_del_mes(gs_65_div_meses[i].month[0],gs_65_year[-1])
        # # Colocamos el último intervalo
        row = contador -1
        gs_65_div_meses[i].at[row, "divs"] = [21, final_day]
        
#Pasamos a tupla la lista de los medidores para eliminar duplicados de cmd y medidores en un determinado periodo
gs_cmd_65.posicion_medidores = gs_cmd_65.posicion_medidores.apply(lambda x: tuple(x))

#Contenedor general del df, para evidencia
gs_65_ROI = pd.DataFrame()

for i in range (0, len(gs_65_months)): # Para recorrer por meses la lista de dataframes
    
    for j in range (0, 3): # El 3, porque son 3 intervalos de tiempo por mes
        
        #Delimitamos fechas, fecha inicial, fecha final, para el filtrado de datos desde gg_cmd_55
        initial_date = dt.date(gs_65_div_meses[i].year[j], gs_65_div_meses[i].month[j], gs_65_div_meses[i].divs[j][0])
        last_date = dt.date(gs_65_div_meses[i].year[j], gs_65_div_meses[i].month[j], gs_65_div_meses[i].divs[j][1])
        #Filtrado del df completo de msgs
        aux_cmd = gs_cmd_65.loc[gs_cmd_65["fecha_registro"].between(initial_date, last_date)]
        aux_cmd = aux_cmd.drop_duplicates(subset=["identificador", "posicion_medidores"],keep="first")
        aux_cmd = aux_cmd.reset_index()
        aux_cmd = aux_cmd.drop(["index"], axis = 1)
        #Concatenamos en un df cada uno de los renglones aux
        gs_65_ROI = pd.concat([gs_65_ROI, aux_cmd])
        
        for k in range (0, len(zonas_gs)):
            
            aux_zonas = pd.DataFrame()
            aux_zonas = aux_cmd[aux_cmd.zona == zonas_gs[k]]
            gs_65_div_meses[i].at[(k + (2*k)) + j, "msgs"] = sum(aux_zonas.Meds_Acum)
            gs_65_div_meses[i].at[(k + (2*k)) + j, "zona"] = zonas_gs[k]
            #gg_cmd_55_zonas[j] = gg_cmd_55_zonas[j].sort_values(['month', 'zona'],ascending=True)

# Variables cruciales:
## gx_cmd_x5_data (general)
## gx_cmd_x5_zonas (x zona)

# Creamos un excel con la información desglosada de los df resumen ROI

In [27]:
writer = pd.ExcelWriter('Gammas_ROI.xlsx')
gg_55_ROI.to_excel(writer, sheet_name="gg_55", index=False)
gg_65_ROI.to_excel(writer, sheet_name="gg_65", index=False)
gs_55_ROI.to_excel(writer, sheet_name="gs_55", index=False)
gs_65_ROI.to_excel(writer, sheet_name="gs_65", index=False)
writer.save()
writer.close()

C:\Users\Cantilever\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


# Exportar la data obtenida a excel

In [28]:
# Para nombrar los worksheets del excel

def number2month(key):
    
    m = {1 : 'Enero',
         2 : 'Febrero',
         3 : 'Marzo',
         4 : 'Abril',
         5 : 'Mayo',
         6 : 'Junio',
         7 : 'Julio',
         8 : 'Agosto',
         9 : 'Septiembre',
         10 : 'Octubre',
         11 : 'Noviembre',
         12 : 'Diciembre'}
    
    try:
        out = m[key]
        return out
    
    except:
        raise ValueError('Not a month')

In [29]:
for i in range (0, len(gg_55_div_meses)): 
    gg_55_div_meses[i] = gg_55_div_meses[i].rename({'msgs': 'msgs_corte'}, axis=1)
    gg_65_div_meses[i] = gg_65_div_meses[i].rename({'msgs': 'msgs_reco'}, axis=1)
    
for i in range (0, len(gs_55_div_meses)): 
    gs_55_div_meses[i] = gs_55_div_meses[i].rename({'msgs': 'msgs_corte'}, axis=1)
    gs_65_div_meses[i] = gs_65_div_meses[i].rename({'msgs': 'msgs_reco'}, axis=1)

# Para n, si funciona, aplicará para n+1

# Aplicar para n+1

In [30]:
# Declaramos nuevas variables contenedoras 

gg_zonas = []
gs_zonas = []

# Para gg

for i in range (0, len(gg_55_div_meses)):
    
    #Se obtiene los comandos y posisiciones en cada mensaje
    gg_55_div_meses[i]["div_inf"] = list(map(lambda x: x[0], gg_55_div_meses[i].divs))
    #Se obtiene los comandos y posisiciones en cada mensaje
    gg_55_div_meses[i]["div_sup"] = list(map(lambda x: x[1], gg_55_div_meses[i].divs))
    
    #Eliminar informacion de no interes
    gg_55_div_meses[i] = gg_55_div_meses[i].drop(["year", "divs"], axis=1)
    gg_55_div_meses[i] = gg_55_div_meses[i].reset_index()
    gg_55_div_meses[i] = gg_55_div_meses[i].drop(["index"], axis= 1)
    
    #Se obtiene los comandos y posisiciones en cada mensaje
    gg_65_div_meses[i]["div_inf"] = list(map(lambda x: x[0], gg_65_div_meses[i].divs))
    #Se obtiene los comandos y posisiciones en cada mensaje
    gg_65_div_meses[i]["div_sup"] = list(map(lambda x: x[1], gg_65_div_meses[i].divs))
    
    #Eliminar informacion de no interes
    gg_65_div_meses[i] = gg_65_div_meses[i].drop(["year", "divs"], axis=1)
    gg_65_div_meses[i] = gg_65_div_meses[i].reset_index()
    gg_65_div_meses[i] = gg_65_div_meses[i].drop(["index"], axis= 1)
    
    
    #Hacemos el cruce, y reordenamos las columnas de los df
    gg_zonas.append(gg_55_div_meses[i].merge(gg_65_div_meses[i], on=["month", "zona", "div_inf", "div_sup"]))
    gg_zonas[i] = gg_zonas[i][['month','zona','div_inf','div_sup','msgs_corte', 'msgs_reco']]
    
# Para gs

for i in range (0, len(gs_55_div_meses)):
    
    #Se obtiene los comandos y posisiciones en cada mensaje
    gs_55_div_meses[i]["div_inf"] = list(map(lambda x: x[0], gs_55_div_meses[i].divs))
    #Se obtiene los comandos y posisiciones en cada mensaje
    gs_55_div_meses[i]["div_sup"] = list(map(lambda x: x[1], gs_55_div_meses[i].divs))
    
    #Eliminar informacion de no interes
    gs_55_div_meses[i] = gs_55_div_meses[i].drop(["year", "divs"], axis=1)
    gs_55_div_meses[i] = gs_55_div_meses[i].reset_index()
    gs_55_div_meses[i] = gs_55_div_meses[i].drop(["index"], axis= 1)
    
    #Se obtiene los comandos y posisiciones en cada mensaje
    gs_65_div_meses[i]["div_inf"] = list(map(lambda x: x[0], gs_65_div_meses[i].divs))
    #Se obtiene los comandos y posisiciones en cada mensaje
    gs_65_div_meses[i]["div_sup"] = list(map(lambda x: x[1], gs_65_div_meses[i].divs))
    
    #Eliminar informacion de no interes
    gs_65_div_meses[i] = gs_65_div_meses[i].drop(["year", "divs"], axis=1)
    gs_65_div_meses[i] = gs_65_div_meses[i].reset_index()
    gs_65_div_meses[i] = gs_65_div_meses[i].drop(["index"], axis= 1)
    
    #Hacemos el cruce, y reordenamos las columnas de los df
    gs_zonas.append(gs_55_div_meses[i].merge(gs_65_div_meses[i], on=["month", "zona", "div_inf", "div_sup"]))
    gs_zonas[i] = gs_zonas[i][['month','zona','div_inf','div_sup','msgs_corte', 'msgs_reco']]

In [31]:
#Medidores y dispositivos

# Añadir cantidad de medidores, y de tarjetas gamma

sim_with_meds = sim.merge(gamma_meds, how='inner', on = ['dispositivo'])
german_with_meds = german.merge(gamma_meds, how='inner', on = ['dispositivo'])

In [32]:
for i in range (0, len(gg_zonas)):
    gg_zonas[i] = gg_zonas[i].assign(dispositivos=None)
    gg_zonas[i] = gg_zonas[i].assign(medidores=None)
    for j in range (0, len(gg_zonas[i])):
        gg_zonas[i].at[j, "dispositivos"] = len(german[german.zona == gg_zonas[i].zona[j]])
        gg_zonas[i].at[j, "medidores"] = len(german_with_meds[german_with_meds.zona == gg_zonas[i].zona[j]])
    
for i in range (0, len(gs_zonas)):
    gs_zonas[i] = gs_zonas[i].assign(dispositivos=None)
    gs_zonas[i] = gs_zonas[i].assign(medidores=None)
    for j in range (0, len(gs_zonas[i])):
        gs_zonas[i].at[j, "dispositivos"] = len(sim[sim.zona == gs_zonas[i].zona[j]])
        gs_zonas[i].at[j, "medidores"] = len(sim_with_meds[sim_with_meds.zona == gs_zonas[i].zona[j]])    

In [33]:
# Añadir condensado de zonas
# Primero gg

#Obtenemos los intervalos inferiores de los dias
intervalo_inf = list(gg_zonas[0].div_inf.unique())

for i in range (len(gg_zonas)):
    
    cortes = []
    recos = []
    dispos = []
    medidores = []

    for j in range (0, len(intervalo_inf)):
        aux = gg_zonas[i][gg_zonas[i]["div_inf"] == intervalo_inf[j]]
        cortes.append(sum(aux.msgs_corte))
        recos.append(sum(aux.msgs_reco))
        dispos.append(sum(aux.dispositivos))
        medidores.append(sum(aux.medidores))
    
    copy_zona = gg_zonas[i][gg_zonas[i].zona == zonas_gg[0]]
    #print(copia_zona)
    gg_zonas[i] = pd.concat([gg_zonas[i], copy_zona], axis = 0, sort = False)
    gg_zonas[i] = gg_zonas[i].reset_index()
    gg_zonas[i] = gg_zonas[i].drop(["index"], axis=1)
    
    filas = len(gg_zonas[i])
    
    for k in range ((filas-3), filas):
        gg_zonas[i].at[k, "zona"] = "DVMS"
        gg_zonas[i].at[k, "msgs_corte"] = cortes[k-15]
        gg_zonas[i].at[k, "msgs_reco"] = recos[k-15]
        gg_zonas[i].at[k, "dispositivos"] = dispos[k-15]
        gg_zonas[i].at[k, "medidores"] = medidores[k-15]
        
# GS

#Obtenemos los intervalos inferiores de los dias
intervalo_inf = list(gs_zonas[0].div_inf.unique())

for i in range (len(gs_zonas)):
    
    cortes = []
    recos = []
    dispos = []
    medidores = []

    for j in range (0, len(intervalo_inf)):
        aux = gs_zonas[i][gs_zonas[i]["div_inf"] == intervalo_inf[j]]
        cortes.append(sum(aux.msgs_corte))
        recos.append(sum(aux.msgs_reco))
        dispos.append(sum(aux.dispositivos))
        medidores.append(sum(aux.medidores))
    
    copy_zona = gs_zonas[i][gs_zonas[i].zona == zonas_gs[0]]
    #print(copia_zona)
    gs_zonas[i] = pd.concat([gs_zonas[i], copy_zona], axis = 0, sort = False)
    gs_zonas[i] = gs_zonas[i].reset_index()
    gs_zonas[i] = gs_zonas[i].drop(["index"], axis=1)
    
    filas = len(gs_zonas[i])
    
    for k in range ((filas-3), filas):
        gs_zonas[i].at[k, "zona"] = "Acapulco"
        gs_zonas[i].at[k, "msgs_corte"] = cortes[k-15]
        gs_zonas[i].at[k, "msgs_reco"] = recos[k-15]
        gs_zonas[i].at[k, "dispositivos"] = dispos[k-15]
        gs_zonas[i].at[k, "medidores"] = medidores[k-15]

In [34]:
# De aquí en adelante, ocuparemos variables
# gg_zonas 
# gs_zonas

# Generación de excel data por zona (9 archivos en total)

data_names_gg = []
data_names_gs = []

for i in range (0, len(gg_zonas)):
    
    name = "gg_data_mes_" + str(gg_zonas[i].month[0]) + ".xlsx"
    data_names_gg.append(name)
    gg_zonas[i].to_excel(name, index=False)
    
for i in range (0, len(gs_zonas)):
    
    name = "gs_data_mes_" + str(gs_zonas[i].month[0]) + ".xlsx"
    data_names_gs.append(name)
    gs_zonas[i].to_excel(name, index=False)

# Hasta aquí, la exportación de la data se llevó con éxito
## Resta la presentación en excel de c/u de estos datos

In [ ]:
# Recapitulando...
# Tenemos varios xlsx para gg y gs con la información condensada de los msgs
# Se deben de crear 2 archivos distintos: uno para gg, y otro para gs
# Paso 1: generar los dos archivos con n hojas según el mes

## Generar el archivo con la data dispersada en hojas para GG

In [40]:
#Importamos libreria
import win32com.client 

#Obtenemos la posición del directorio actual
actual_dir = os.getcwd()
path1 = actual_dir + "\\ROI_GG_Template.xlsx" #Referenciamos archivo template
new_file = actual_dir + "\\NewFile.xlsx" #Nombre de nuevo archivo de interes

#Abrimos la api de excel
office = win32com.client.Dispatch("Excel.Application") 
office.Visible = False #Deshabilitamos la función para no ver excel
wb = office.Workbooks.Open(path1) #Abrimos nuestro archivo template
wb.SaveAs(Filename:=new_file) #Guardamos con un nuevo nombre
wb.Close() #Cerramos el archivo
office.Quit() #Cerramos la API

#Ahora, replicar este algoritmo con n páginas en donde se encuentra la info

In [45]:
data_names_gg

['gg_data_mes_4.xlsx',
 'gg_data_mes_5.xlsx',
 'gg_data_mes_6.xlsx',
 'gg_data_mes_7.xlsx',
 'gg_data_mes_8.xlsx']

In [ ]:
#Test para copiar la data de gg en el nuevo template

#No se manda a llamar a la libreria, esta referenciada al inicio del código
#Obtener la ruta del directorio actual
actual_dir = os.getcwd()
#Referenciar la ruta del template
path_template = actual_dir + "\\ROI_GG_Template.xlsx" #Default



In [47]:
for i in data_names_gg:
    print(i)

gg_data_mes_4.xlsx
gg_data_mes_5.xlsx
gg_data_mes_6.xlsx
gg_data_mes_7.xlsx
gg_data_mes_8.xlsx


In [ ]:
#Copiar la data de los distintos archivos

# Para lanzar Excel
xl = Dispatch("Excel.Application")
xl.Visible = False

wb1 = xl.Workbooks.Open(Filename=r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\gg_template.xlsx')
wb2 = xl.Workbooks.Open(Filename=r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\gg_data_mes_5.xlsx')

#Copiar ambas sheets del template al archivo de data
ws1 = wb1.Worksheets(1)
ws1.Copy(Before=wb2.Worksheets(1))
ws1 = wb1.Worksheets(2)
ws1.Copy(Before=wb2.Worksheets(2))

#Cerramos y guardamos archivo. Quitamos excel
wb2.Close(SaveChanges=True)
xl.Quit()

# Primero para gg

In [36]:
actual_dir = os.getcwd()

In [37]:
# Para lanzar Excel
xl = Dispatch("Excel.Application")
xl.Visible = False

#Ruta del template
path1 = actual_dir + '\\gg_template.xlsx'

#Ruta de la data
path2 = actual_dir + '\\gg_data_mes_5.xlsx'

#Abrimos el archivo del template, en la hoja de la data
wb1 = xl.Workbooks.Open(Filename=path1)
ws1 = wb1.Worksheets("Data")

#Abrimos el archivo de la data, en la hoja de la data
wb2 = xl.Workbooks.Open(Filename=path2)
ws2 = wb2.Worksheets("Sheet1")

#Realizaremos la copia de la data generada en el template
for i in range(1, 9):
    for j in range(1, 20):
        ws1.cell(row=i,column=j).value = ws2.cell(row=i,column=j).value

#Cerramos el wb de la data
wb2.Close(SaveChanges=True)

#Cambiar nombre de las hojas y guardar

ws1.title = "Data_gg_5"
ws3 = wb1.Worksheets("Template")
ws3.title = "MAYO"

#Cerramos y guardamos con un nuevo nombre
wb1.save("Ejemplo.xlsx")

#Cerramos archivo templante sin guardar
wb1.Close(SaveChanges=False)

#Cerramos excel
xl.Quit()

AttributeError: '<win32com.gen_py.Microsoft Excel 16.0 Object Library._Worksheet instance at 0x2102432320912>' object has no attribute 'cell'

In [51]:
# Para lanzar Excel
xl = Dispatch("Excel.Application")
xl.Visible = False

wb1 = xl.Workbooks.Open(Filename=r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\gg_template.xlsx')
wb2 = xl.Workbooks.Open(Filename=r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\gg_data_mes_5.xlsx')

#Copiar ambas sheets del template al archivo de data
ws1 = wb1.Worksheets(1)
ws1.Copy(Before=wb2.Worksheets(1))
ws1 = wb1.Worksheets(2)
ws1.Copy(Before=wb2.Worksheets(2))

#Cerramos y guardamos archivo. Quitamos excel
wb2.Close(SaveChanges=True)
xl.Quit()

In [ ]:
for i in range(1, 9):
    for j in range(1, 20):
        sheet1.cell(row=i,column=j).value = sheet.cell(row=i,column=j).value
wb1.save('file2.xlsx')

In [35]:
# Para eliminar la data pasada (no sobreescribir datos,  y se genere una sheet con nombre de copia)
# Consideramos que no sería la primera ocasión en que se corra el script
# Por lo que deberían existir las hojas de data y copias de templates 

# Primero se eliminará la data

path1 = r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\ROI_Gamma_German_4.1.xlsx'

xl = Dispatch("Excel.Application")
xl.Visible = False 
xl.DisplayAlerts=False

#Borramos primera hoja
wb1 = xl.Workbooks.Open(Filename=path1)
ws1 = wb1.Worksheets("gg_55").Delete()
#Borramos segunda hoja
ws1 = wb1.Worksheets("gg_65").Delete()

wb1.Close(SaveChanges=True)
xl.Quit()

In [36]:
#Leemos la data del txt

datos = []
with open("meses_roi.txt") as fname:
	lineas = fname.readlines()
	for linea in lineas:
		datos.append(linea.strip('\n'))

# Procedemos a eliminar las hojas en donde se encuentran los ROI de n meses

path1 = r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\ROI_Gamma_German_4.1.xlsx'

xl = Dispatch("Excel.Application")
xl.Visible = False 
xl.DisplayAlerts=False

#Borramos primera hoja
wb1 = xl.Workbooks.Open(Filename=path1)

for i in range(0, len(datos)):
    wb1.Worksheets(datos[i]).Delete()
    
wb1.Close(SaveChanges=True)
xl.Quit()

In [37]:
# Tomar del archivo base "Data_GG", para pasarlo al template de ROI
# Se rellena la data de nuevo en el archivo

path1 = r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\Data_GG.xlsx'
path2 = r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\ROI_Gamma_German_4.1.xlsx'

xl = Dispatch("Excel.Application")
xl.Visible = False  # You can remove this line if you don't want the Excel application to be visible

wb1 = xl.Workbooks.Open(Filename=path1)
wb2 = xl.Workbooks.Open(Filename=path2)

ws1 = wb1.Worksheets(1)
ws1.Copy(Before=wb2.Worksheets(1))

ws1 = wb1.Worksheets(2)
ws1.Copy(Before=wb2.Worksheets(2))

wb2.Close(SaveChanges=True)
xl.Quit()

In [38]:
month_worksheet = list(GG65_Sheet.month.unique())
months = []

# Ruta del template
path1 = r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\ROI_Gamma_German_4.1.xlsx'
xl = Dispatch("Excel.Application")
xl.Visible = False
wb1 = xl.Workbooks.Open(Filename=path1)
ws1 = wb1.Worksheets("Template")

for i in range (0, len(month_worksheet)):
    ws1.Copy(Before=wb1.Worksheets(1))
    wb1.Worksheets(1).Name = number2month(month_worksheet[i])+"_2022"
    months.append(number2month(month_worksheet[i])+"_2022")
    
wb1.Close(SaveChanges=True)
#Cerramos excel
xl.Quit()

NameError: name 'GG65_Sheet' is not defined

In [ ]:
# Sobreescribimos el archivo txt de las worksheets
with open('meses_roi.txt', 'w+') as f: 
    for items in months: 
        f.write('%s\n' %items) 
    f.close()

## Hasta aquí va la parte de automatización

## Divivir en 24 hrs, semana y mes

In [ ]:
# #Para las ultimas 24 hrs
# gg55_24hrs=gammas_german_corte_55.loc[gammas_german_corte_55["fecha_registro"].between(dt.date.today() - dt.timedelta(days = 1), dt.date.today())]
# gg65_24hrs=gammas_german_reco_65.loc[gammas_german_reco_65["fecha_registro"].between(dt.date.today() - dt.timedelta(days = 1), dt.date.today())]
# gs55_24hrs=gammas_sim_corte_55.loc[gammas_sim_corte_55["fecha_registro"].between(dt.date.today() - dt.timedelta(days = 1), dt.date.today())]
# gs65_24hrs=gammas_sim_reco_65.loc[gammas_sim_reco_65["fecha_registro"].between(dt.date.today() - dt.timedelta(days = 1), dt.date.today())]
# 
# #De la ultima semana
# gg55_7dias=gammas_german_corte_55.loc[gammas_german_corte_55["fecha_registro"]>=dt.date.today()-dt.timedelta(days=7)]
# gg65_7dias=gammas_german_reco_65.loc[gammas_german_reco_65["fecha_registro"]>=dt.date.today()-dt.timedelta(days=7)]
# gs55_7dias=gammas_sim_corte_55.loc[gammas_sim_corte_55["fecha_registro"]>=dt.date.today()-dt.timedelta(days=7)]
# gs65_7dias=gammas_sim_reco_65.loc[gammas_sim_reco_65["fecha_registro"]>=dt.date.today()-dt.timedelta(days=7)]
# 
# #Del ultimo mes
# gg55_30dias=gammas_german_corte_55.loc[gammas_german_corte_55["fecha_registro"]>=dt.date.today()-dt.timedelta(days=30)]
# gg65_30dias=gammas_german_reco_65.loc[gammas_german_reco_65["fecha_registro"]>=dt.date.today()-dt.timedelta(days=30)]
# gs55_30dias=gammas_sim_corte_55.loc[gammas_sim_corte_55["fecha_registro"]>=dt.date.today()-dt.timedelta(days=30)]
# gs65_30dias=gammas_sim_reco_65.loc[gammas_sim_reco_65["fecha_registro"]>=dt.date.today()-dt.timedelta(days=30)]

# cmds_tiempo=[[sum(gg55_24hrs["Meds_Acum"]),sum(gg55_7dias["Meds_Acum"]),sum(gg55_30dias["Meds_Acum"])],
#  [sum(gg65_24hrs["Meds_Acum"]),sum(gg65_7dias["Meds_Acum"]),sum(gg65_30dias["Meds_Acum"])],
#  [sum(gs55_24hrs["Meds_Acum"]),sum(gs55_7dias["Meds_Acum"]),sum(gs55_30dias["Meds_Acum"])],
#  [sum(gs65_24hrs["Meds_Acum"]),sum(gs65_7dias["Meds_Acum"]),sum(gs65_30dias["Meds_Acum"])]]

# columnas = ['24hrs', '7_dias', '30_dias'] # definimos los nombres de las columnas
# filas = ['GammaGerman_CMD_55', 'GammaGerman_CMD_65', 'GammaSim_CMD_55', 'GammaSim_CMD_65'] # definimos los nombres de las filas
# 
# comunicacion = pd.DataFrame(cmds_tiempo, columns=columnas, index=filas)
# comunicacion

# print(len(gammas_german_corte_55))
# print(len(gammas_german_reco_65))
# print(len(gammas_sim_corte_55))
# print(len(gammas_sim_reco_65))

# sim_with_meds = sim.merge(gamma_meds, how='inner', on = ['dispositivo'])

# german_with_meds = german.merge(gamma_meds, how='inner', on = ['dispositivo'])

# Generación de pdf ROI

# import jinja2
# import pdfkit
# import datetime as dt
# import win32com.client as win32

# def current_date_format(date):
#     
#     months = ("Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre")
#     day = date.day
#     month = months[date.month - 1]
#     year = date.year
#     messsage = "{} de {} del {}".format(day, month, year)
#     return messsage
# 
# 
# fecha = dt.date.today()
# fecha = current_date_format(fecha)

In [ ]:
# corte_24hrs = sum(gg55_24hrs["Meds_Acum"]) + sum(gs55_24hrs["Meds_Acum"])
# reco_24hrs = sum(gg65_24hrs["Meds_Acum"]) + sum(gs65_24hrs["Meds_Acum"])
# corte_semana = sum(gg55_7dias["Meds_Acum"]) + sum(gs55_7dias["Meds_Acum"])
# reco_semana = sum(gg65_7dias["Meds_Acum"]) + sum(gs65_7dias["Meds_Acum"])
# corte_mes = sum(gg55_30dias["Meds_Acum"]) + sum(gs55_30dias["Meds_Acum"])
# reco_mes = sum(gg65_30dias["Meds_Acum"]) + sum(gs65_30dias["Meds_Acum"])
# dispositivos_gg = len(german)
# dispositivos_gs = len(sim)
# medidores_gg = len(german_with_meds)
# medidores_gs = len(sim_with_meds)
# 
# excel = win32.gencache.EnsureDispatch('Excel.Application')
# wb = excel.Workbooks.Open(r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\ROI_GRAL.xlsx')
# # Alternately, specify the full path to the workbook
# # wb = excel.Workbooks.Open(r'C:\myfiles\excel\workbook2.xlsx')
# excel.Visible = False
# ws = wb.Worksheets("Gral")
# 
# ws.Cells(11,4).Value = dispositivos_gs
# ws.Cells(14,4).Value = medidores_gs
# ws.Cells(9,11).Value = corte_24hrs
# ws.Cells(9,14).Value = reco_24hrs
# ws.Cells(13,11).Value = corte_semana
# ws.Cells(13,14).Value = reco_semana
# ws.Cells(17,11).Value = corte_mes
# ws.Cells(17,14).Value = reco_mes
# ws.Cells(2,12).Value = fecha
# 
# wb.Save()
# excel.Application.Quit()

In [38]:
# for i in range (0, len(zonas_gg)):
#     
#     excel = win32.gencache.EnsureDispatch('Excel.Application')
#     wb = excel.Workbooks.Open(r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\ROI_Gamma_GERMAN_2.0.xlsx')
#     # Alternately, specify the full path to the workbook
#     # wb = excel.Workbooks.Open(r'C:\myfiles\excel\workbook2.xlsx')
#     excel.Visible = False
#     ws = wb.Worksheets(zonas_gg[i])
#     
#     ws.Cells(10,4).Value = len(german[german.zona==zonas_gg[i]])
#     ws.Cells(13,4).Value = len(german_with_meds[german_with_meds.zona==zonas_gg[i]])
#     ws.Cells(9,11).Value = sum(gg55_24hrs[gg55_24hrs.zona==zonas_gg[i]]["Meds_Acum"])
#     ws.Cells(9,14).Value = sum(gg65_24hrs[gg65_24hrs.zona==zonas_gg[i]]["Meds_Acum"])
#     ws.Cells(13,11).Value = sum(gg55_7dias[gg55_7dias.zona==zonas_gg[i]]["Meds_Acum"])
#     ws.Cells(13,14).Value = sum(gg65_7dias[gg65_7dias.zona==zonas_gg[i]]["Meds_Acum"])
#     ws.Cells(17,11).Value = sum(gg55_30dias[gg55_30dias.zona==zonas_gg[i]]["Meds_Acum"])
#     ws.Cells(17,14).Value = sum(gg65_30dias[gg65_30dias.zona==zonas_gg[i]]["Meds_Acum"])
#     ws.Cells(2,12).Value = fecha
#     
#     wb.Save()
# 
# ws = wb.Worksheets("GENERAL")
# ws.Cells(2,12).Value = fecha
# 
# wb.Save()
# wb.Close(True) 
# map(lambda book: book.Close(False), excel.Workbooks)
# excel.Quit()

# for i in range (0, len(zonas_gs)):
#     
#     excel = win32.gencache.EnsureDispatch('Excel.Application')
#     wb = excel.Workbooks.Open(r'C:\Users\Cantilever\Desktop\Gammas\Roi_Gamma\ROI_Gamma_SIM_2.0.xlsx')
#     # Alternately, specify the full path to the workbook
#     # wb = excel.Workbooks.Open(r'C:\myfiles\excel\workbook2.xlsx')
#     excel.Visible = False
#     ws = wb.Worksheets(zonas_gs_prima[i])
#     
#     ws.Cells(10,4).Value = len(sim[sim.zona==zonas_gs[i]])
#     ws.Cells(13,4).Value = len(sim_with_meds[sim_with_meds.zona==zonas_gs[i]])
#     ws.Cells(9,11).Value = sum(gs55_24hrs[gs55_24hrs.zona==zonas_gs[i]]["Meds_Acum"])
#     ws.Cells(9,14).Value = sum(gs65_24hrs[gs65_24hrs.zona==zonas_gs[i]]["Meds_Acum"])
#     ws.Cells(13,11).Value = sum(gs55_7dias[gs55_7dias.zona==zonas_gs[i]]["Meds_Acum"])
#     ws.Cells(13,14).Value = sum(gs65_7dias[gs65_7dias.zona==zonas_gs[i]]["Meds_Acum"])
#     ws.Cells(17,11).Value = sum(gs55_30dias[gs55_30dias.zona==zonas_gs[i]]["Meds_Acum"])
#     ws.Cells(17,14).Value = sum(gs65_30dias[gs65_30dias.zona==zonas_gs[i]]["Meds_Acum"])
#     ws.Cells(2,12).Value = fecha
#     
#     wb.Save()
#     
# ws = wb.Worksheets("GENERAL")
# ws.Cells(2,12).Value = fecha
# 
# wb.Save()
# wb.Close(True) 
# map(lambda book: book.Close(False), excel.Workbooks)
# excel.Quit()

# Algoritmo para obtener la parte de las graficas

In [ ]:
# Renombrar variables para su analisis mas sencillo
# Identificar en los 4 df principales los indices de filas con datos de demos
# Guardar en una variable dichos indices
# Eliminar de los df


#Se realiza solo con gg, gs no tiene datos de demos

#Renombrar variables de datos
gg_cmd_55 = gammas_german_corte_55
gg_cmd_55["month"] = gg_cmd_55.fecha.dt.month
gg_cmd_55["year"] = gg_cmd_55.fecha.dt.year

gg_cmd_65 = gammas_german_reco_65
gg_cmd_65["month"] = gg_cmd_65.fecha.dt.month
gg_cmd_65["year"] = gg_cmd_65.fecha.dt.year

gs_cmd_55 = gammas_sim_corte_55
gs_cmd_55["month"] = gs_cmd_55.fecha.dt.month
gs_cmd_55["year"] = gs_cmd_55.fecha.dt.year

gs_cmd_65 = gammas_sim_reco_65
gs_cmd_65["month"] = gs_cmd_65.fecha.dt.month
gs_cmd_65["year"] = gs_cmd_65.fecha.dt.year

print(len(gg_cmd_55))
print(len(gg_cmd_65))

#Identificamos los indices de los datos demo
demo_gg_55 = list(np.where(gg_cmd_55.zona == "DEMO"))
ermita_gg_55 = list(np.where(gg_cmd_55.zona == "ERMITA"))

demo_gg_65 = list(np.where(gg_cmd_65.zona == "DEMO"))
ermita_gg_65 = list(np.where(gg_cmd_65.zona == "ERMITA"))

#Eliminamos del df los datos de no interes
gg_cmd_55.drop(gg_cmd_55.index[demo_gg_55],0,inplace=True)
gg_cmd_55.drop(gg_cmd_55.index[ermita_gg_55],0,inplace=True)

gg_cmd_65.drop(gg_cmd_65.index[demo_gg_65],0,inplace=True)
gg_cmd_65.drop(gg_cmd_65.index[ermita_gg_65],0,inplace=True)

#Reseteamos indices, borramos columna de indices
gg_cmd_55 = gg_cmd_55.reset_index()
gg_cmd_65 = gg_cmd_65.reset_index()

gg_cmd_55 = gg_cmd_55.drop(["index"], axis=1)
gg_cmd_65 = gg_cmd_65.drop(["index"], axis=1)

print(len(gg_cmd_55))
print(len(gg_cmd_65))

In [ ]:
#Aplicar un filtro para que segmente datos dado mes y año
#Se necesita historico por zona y general, para cada gamma

#Primero obtendremos por zona, para sumarlos al general (y no contabilizar demos, etc)
#Creamos una lista para tener los datos segmentados por zonas
gg_cmd_55_zonas = []
gg_cmd_65_zonas = []
gs_cmd_55_zonas = []
gs_cmd_65_zonas = []

#Obtenemos las diferentes zonas para cada gamma

In [ ]:
gg_zonas = gg_cmd_55.zona.unique()
gs_zonas = gs_cmd_55.zona.unique()

In [ ]:
gg_zonas

In [ ]:
gs_zonas

In [ ]:
for i in gg_zonas:
    
    aux = gg_cmd_55[gg_cmd_55["zona"] == i]
    aux = aux.reset_index()
    aux.drop(["index"],axis=1,inplace=True)
    gg_cmd_55_zonas.append(aux)
    aux=[]
    
    aux = gg_cmd_65[gg_cmd_65["zona"] == i]
    aux = aux.reset_index()
    aux.drop(["index"],axis=1,inplace=True)
    gg_cmd_65_zonas.append(aux)
    aux=[]
    
for i in gs_zonas:
    
    aux = gs_cmd_55[gs_cmd_55["zona"] == i]
    aux = aux.reset_index()
    aux.drop(["index"],axis=1,inplace=True)
    gs_cmd_55_zonas.append(aux)
    aux=[]
    
    aux = gs_cmd_65[gs_cmd_65["zona"] == i]
    aux = aux.reset_index()
    aux.drop(["index"],axis=1,inplace=True)
    gs_cmd_65_zonas.append(aux)
    aux=[]

In [ ]:
gg_cmd_55_zonas[0]

In [ ]:
# Se encuentra dividido por zonas, lo que resta es comenzar a dividir por 

In [ ]:
aux = aux[(aux["month"] >= 4) & (aux["year"] >= 2022)]
data = []

In [ ]:
aux

In [ ]:
aux_2 = aux[(aux["month"] == 4)]

In [ ]:
sum(aux_2.Meds_Acum)

In [ ]:
# Primero se ejecutara de manera semi automatica

